In [2]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"
#r "nuget: Microsoft.ML, 3.0.1"
#r "nuget: Microsoft.ML.FastTree, 3.0.1"
#r "nuget: Microsoft.Extensions.ML, 3.0.1"

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.Extensions.ML, 3.0.1 Microsoft.ML, 3.0.1 Microsoft.ML.FastTree, 3.0.1 Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha

In [15]:
using Microsoft.ML;
using Microsoft.ML.Data;
using System.IO;
using System.Linq;

public class MLBBaseballBatter
{
    [LoadColumn(0), ColumnName("InductedToHallOfFame")]
    public bool InductedToHallOfFame { get; set; }

    [LoadColumn(1), ColumnName("OnHallOfFameBallot")]
    public bool OnHallOfFameBallot { get; set; }

    [LoadColumn(2), ColumnName("FullPlayerName")]
    public string FullPlayerName { get; set; }

    [LoadColumn(3), ColumnName("YearsPlayed")]
    public float YearsPlayed { get; set; }

    [LoadColumn(4), ColumnName("AB")]
    public float AB { get; set; }

    [LoadColumn(5), ColumnName("R")]
    public float R { get; set; }

    [LoadColumn(6), ColumnName("H")]
    public float H { get; set; }

    [LoadColumn(7), ColumnName("Doubles")]
    public float Doubles { get; set; }

    [LoadColumn(8), ColumnName("Triples")]
    public float Triples { get; set; }

    [LoadColumn(9), ColumnName("HR")]
    public float HR { get; set; }

    [LoadColumn(10), ColumnName("RBI")]
    public float RBI { get; set; }

    [LoadColumn(11), ColumnName("SB")]
    public float SB { get; set; }

    [LoadColumn(12), ColumnName("BattingAverage")]
    public float BattingAverage { get; set; }

    [LoadColumn(13), ColumnName("SluggingPct")]
    public float SluggingPct { get; set; }

    [LoadColumn(14), ColumnName("AllStarAppearances")]
    public float AllStarAppearances { get; set; }

    [LoadColumn(15), ColumnName("TB")]
    public float TB { get; set; }

    [LoadColumn(16), ColumnName("TotalPlayerAwards")]
    public float TotalPlayerAwards { get; set; }

    [LoadColumn(17), ColumnName("LastYearPlayed")]
    public float LastYearPlayed { get; set; }

    [LoadColumn(18), ColumnName("ID")]
    public string ID { get; set; }

    public override string ToString()
    {
        var mlbBatterString = $@"FullPlayerName:{this.FullPlayerName}, YearPlayed:{this.YearsPlayed}, AB:{this.AB}
        R:{this.R}, H:{this.H}, Doubles:{this.Doubles}, Triples:{this.Triples}, HR:{this.HR}, RBI:{this.RBI}, SB:{this.SB},
        BattingAvg:{this.BattingAverage}, Slg:{this.SluggingPct}, AllStar:{this.AllStarAppearances},
        TB:{this.TB}, LastYearPlayed:{this.LastYearPlayed}, ID:{this.ID}";

        return mlbBatterString;
    }

    public static MLBBaseballBatter FromCsv(string csvLine)
    {
        string[] values = csvLine.Split(',');
        MLBBaseballBatter mlbBaseballBatter = new MLBBaseballBatter();

        mlbBaseballBatter.InductedToHallOfFame = Convert.ToBoolean(values[0]);
        mlbBaseballBatter.OnHallOfFameBallot = Convert.ToBoolean(values[1]);
        mlbBaseballBatter.FullPlayerName = Convert.ToString(values[2]);
        mlbBaseballBatter.YearsPlayed = float.Parse(values[3], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.AB = float.Parse(values[4], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.R = float.Parse(values[5], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.H = float.Parse(values[6], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.Doubles = float.Parse(values[7], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.Triples = float.Parse(values[8], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.HR = float.Parse(values[9], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.RBI = float.Parse(values[10], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.SB = float.Parse(values[11], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.BattingAverage = float.Parse(values[12], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.SluggingPct = float.Parse(values[13], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.AllStarAppearances = float.Parse(values[14]);
        mlbBaseballBatter.TB = float.Parse(values[15], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.TotalPlayerAwards = float.Parse(values[16], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.LastYearPlayed = float.Parse(values[17], System.Globalization.NumberStyles.Any);
        mlbBaseballBatter.ID = Convert.ToString(values[18]);

        return mlbBaseballBatter;
    }
}

public class MLBHOFPrediction
{
    [ColumnName("PredictedLabel")]
    public bool Prediction { get; set; }

    [ColumnName("Probability")]
    public float Probability { get; set; }

    [ColumnName("Score")]
    public float Score { get; set; }
}

In [23]:
// Directory of the historical baseball data
string filePathMLBBaseballBatters = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Data/MLBBaseballBattersPositionPlayers.csv";
// Read the batters data from the CSV file
var batters = File.ReadAllLines(filePathMLBBaseballBatters)
                        .Skip(1)
                        .Select(v => MLBBaseballBatter.FromCsv(v))
                        .ToList();

// Machine Learing Model and Prediction Engine                       
string modelPath = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Models/InductedToHoF-GeneralizedAdditiveModels.mlnet";
var mlContext = new MLContext(seed: 100);
ITransformer loadedModel;
DataViewSchema schema;

using (var stream = File.OpenRead(modelPath))
{
    loadedModel = mlContext.Model.Load(stream, out schema);
}
TransformerChain<ITransformer> transfomerChain = (TransformerChain<ITransformer>)loadedModel;

var predEngineInductedToHallOfFame = mlContext.Model.CreatePredictionEngine<MLBBaseballBatter, MLBHOFPrediction>(transfomerChain);

// Predict if Mike Trout will be inducted to the Hall of Fame
var mikeTroutStats = batters.Where(b => b.FullPlayerName == "Mike Trout").FirstOrDefault();
var prediction = predEngineInductedToHallOfFame.Predict(mikeTroutStats);
Console.WriteLine(prediction.Probability);

0.69909585
